# Supplementary - Effect of noise on the geometric cutoff point in the optic tectum

Analyses related to Supplementary Figure S9.

In [ ]:
import sys
sys.path.append("..")

from main import *
from zebrafish import *

from scipy.stats import zscore, spearmanr
from scipy.stats import percentileofscore
from brainsmash.mapgen.base import Base

# Setting paths

In [ ]:
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'

atlas = Mapzebrain(atlas_path)
mask_tectum = atlas.get_region_mask(22)
mask_tectum[:, :, :284] = 0

top_directory = '/media/anleg84/KINGSTON1/Datasets/Geometry/'

datasets = identify_folders(top_directory, ['920nm'])

# Computing eigenmode-gradient correlation with noise

In [ ]:
def compute_noise_from_data():
    """
    WARNING: Uses global variables defined outside the function. This function loops over all folders containing calcium imaging data, loading the tectum
    neuronal time series and evaluating the standard deviation of each neuron. Returns a list of std vectors from different larvae.
    """

    timeseries_std = []
    
    for i, folder in enumerate(datasets):
    
        data = load_data(folder)
        centroids = data['centroids_atlas']
        in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
        dff_tectum = data['timeseries'][in_tectum]
        dff_tectum = compute_dff_using_minfilter(dff_tectum, window=120, sigma1=3, sigma2=60)

        timeseries_std.append(np.std(dff_tectum, axis=1))

    return timeseries_std

In [ ]:
def compute_correlations_from_data(sigma=0, smooth_nodes=False, noise_std=0):
    """
    WARNING: Uses global variables defined outside the function. This function loops over all folders containing calcium imaging data, adds noise
    to the time series, then maps neurons to tectal nodes before computing pairwise correlations between nodes. Returns FC matrices.
    """

    FC_matrices = []
    
    for i, folder in enumerate(datasets):
    
        data = load_data(folder)
        centroids = data['centroids_atlas']
        
        in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
        centroids_tectum = centroids[in_tectum]
        dff_tectum = data['timeseries'][in_tectum]
        dff_tectum = compute_dff_using_minfilter(dff_tectum, window=120, sigma1=3, sigma2=60)
        dff_tectum = dff_tectum + np.random.normal(0, noise_std, dff_tectum.shape)
        dff_tectum = filter_timeseries(dff_tectum, 2)

        if smooth_nodes == False:
            if sigma != 0:
                dff_tectum = spatial_smoothing(dff_tectum, centroids_tectum, sigma=sigma)
    
        nn, _ = find_nearest_neighbors(centroids_tectum, nodes_tectum)
        node_timeseries = np.zeros((nodes_tectum.shape[0], dff_tectum.shape[1]))
        for i in range(nodes_tectum.shape[0]):
            neurons_in_node = (nn == i) # Neurons whose nearest neighbor corresponds to the tectum node i
            if np.any(neurons_in_node):
                if np.sum(neurons_in_node) == 1:
                    node_timeseries[i] = dff_tectum[neurons_in_node]
                else:
                    node_timeseries[i] = np.mean(dff_tectum[neurons_in_node], axis=0)

        if smooth_nodes:
            if sigma != 0:
                node_timeseries = spatial_smoothing(node_timeseries, nodes_tectum, sigma=sigma)   
            
        FC = np.corrcoef(node_timeseries)
        FC_matrices.append(FC)
    
    FC_matrices = np.stack(FC_matrices, axis=0)
    FC = np.nanmean(FC_matrices[:], axis=0)
    FC[np.diag_indices(FC.shape[0])] = 0
    
    return FC, FC_matrices

#### Evaluating noise level in data

In [ ]:
timeseries_std = compute_noise_from_data()

In [ ]:
timeseries_std =  np.concatenate(timeseries_std) # Making one distribution out of all the neurons

Plotting std distribution

In [ ]:
plt.hist(timeseries_std, bins=30, density=True)
plt.xlabel('Fluorescence standard deviation')
plt.ylabel('Density')
plt.show()

In [ ]:
std = np.mean(timeseries_std)

Taking the average standard deviation from all neurons.

#### Computing eigenmode-gradient correlations for various noise levels

In [ ]:
# Loading tectum mask and nodes
mask_tectum = atlas.get_region_mask(22)
mask_tectum[:, :, :284] = 0
nodes_tectum = np.load('../Files/nodes_tectum_right_sampled.npy')

In [ ]:
similarity_matrices = []

noise_levels = std * np.linspace(0.5, 5, 10, endpoint=True) # Multiples of the average neuronal standard deviation

for noise in tqdm(noise_levels):

    # Repeating the noise addition process 10 times
    similarity_matrices_ = []
    for _ in range(10):
        C, _ = compute_correlations_from_data(noise_std=noise)
    
        N_modes = C.shape[0]
        gradients, _ = diffusion_mapping(np.abs(C), n_components=N_modes)
        
        vertices = np.load('../Files/tectum_vertices_right.npy') * 40
        vertices = np.stack([vertices[:, 2], vertices[:, 1], vertices[:, 0]], axis=1)
        eigenmodes = np.load('../Files/tectum_eigenmodes_right.npy')[1:]
    
        d = compute_distances(vertices, nodes_tectum)
        eigenmodes = eigenmodes[:, np.argmin(d, axis=0)]
        gradients, _ = diffusion_mapping(np.abs(C), n_components=C.shape[0])
        
        mode_similarity = compute_mode_similarity_matrix(eigenmodes[:30], gradients.T[:30])
        
        similarity_matrices_.append(mode_similarity)

    similarity_matrices.append(similarity_matrices_)

In [ ]:
np.save('../Results/supp_similarity_matrices_noise.npy', similarity_matrices)

Plotting eigenmode-gradient correlation matrices for varying noise levels

In [ ]:
for m in similarity_matrices:
    plt.figure(figsize=(5, 5), dpi=150)
    plt.imshow(np.abs(m[0]), cmap='Reds', vmin=0.1, vmax=1)
    plt.ylabel('Geometric modes')
    plt.xlabel('Functional gradients')

#### Influence of noise on the average eigenmode-gradient correlations

In [ ]:
similarity_matrices = np.load('../Results/supp_similarity_matrices_noise.npy')

noise_levels = std * np.linspace(0.5, 5, 10, endpoint=True)

In [ ]:
r_curve = []
for m in similarity_matrices:
    r_curve_ = []
    for m_ in m:
        r_curve_.append(np.mean(np.diag(np.abs(m_))))
    r_curve.append(r_curve_)
r_curve = np.stack(r_curve, axis=1)

In [ ]:
plt.imshow(r_curve)

Plotting average eigenmode-gradient correlation for various noise levels. Each black dot reflects one replicate of noise addition.

In [ ]:
plt.plot(noise_levels, np.mean(r_curve, axis=0), color='black')
for i in range(r_curve.shape[1]):
    r_values = r_curve[:, i]
    plt.scatter([noise_levels[i]] * len(r_values), r_values, color='black')

plt.axvline(np.mean(timeseries_std))

#### Influence of noise on identification of the geometric cutoff point

In [ ]:
from scipy.optimize import curve_fit

def compute_r_squared(y_data, y_model):
    ss_res = np.sum((y_data - y_model) ** 2)  # Residual sum of squares
    ss_tot = np.sum((y_data - np.mean(y_data)) ** 2)  # Total sum of squares
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

def piecewise_linear(x, x_break, slope, intercept):
    y1 = slope * (x - x_break) + intercept
    y2 = intercept
    return np.where(x < x_break, y1, y2)

def identify_mode_cutoff_from_matrix_diagonal(mode_similarity):
    diagonal = np.diag(np.abs(mode_similarity))
    x = np.arange(len(diagonal))
    y = diagonal
    initial_guess = [30, -1, 0]
    params, _ = curve_fit(piecewise_linear, x, y, p0=initial_guess)
    cutoff = int(np.round(params[0]))
    return cutoff

In [ ]:
cutoff_points = []
for m in similarity_matrices:
    cutoff_points_ = []
    for m_ in m:
        cutoff_points_.append(identify_mode_cutoff_from_matrix_diagonal(m_))
    cutoff_points.append(cutoff_points_)
cutoff_points = np.stack(cutoff_points, axis=1)

In [ ]:
wavelengths = np.load('../Results/wavelengths_tectum.npy')

In [ ]:
cutoff_wavelengths = np.zeros(cutoff_points.shape)
for i in range(cutoff_points.shape[0]):
    for j in range(cutoff_points.shape[1]):
        cutoff_wavelengths[i, j] = wavelengths[cutoff_points[i, j]]

Plotting cutoff wavelengths vs noise level.

In [ ]:
plt.plot(noise_levels, np.mean(cutoff_wavelengths, axis=0))
for i in range(cutoff_wavelengths.shape[1]):
    values = cutoff_wavelengths[:, i]
    plt.scatter([noise_levels[i]] * len(values), values, color='black')

In [ ]:
print(np.mean(cutoff_wavelengths[:, :3].flatten()), np.std(cutoff_wavelengths[:, :3].flatten()))

In [ ]:
print(np.mean(cutoff_wavelengths.flatten()), np.std(cutoff_wavelengths.flatten()))

Verifying over which range the change in cutoff point is non significant.

In [ ]:
for i in range(3, 10):
    print(f'Number of points: {i}')
    data = []
    for j in range(i):
        data.append(cutoff_wavelengths[:, j])
    groups_ANOVA_Tukey(data)

Observing no group-wise difference in cutoff wavelengths for the first 6 levels of noise added to the data (3 standard deviations).

# Rendering figure

In [ ]:
folder = datasets[0]
    
data = load_data(folder)
centroids = data['centroids_atlas']
in_tectum = mask_tectum[centroids[:, 2], centroids[:, 1], centroids[:, 0]] > 0
dff_tectum = data['timeseries'][in_tectum]
dff_tectum = compute_dff_using_minfilter(dff_tectum, window=120, sigma1=3, sigma2=60)
order = np.flip(np.argsort(autocorr_timeseries(dff_tectum))) # Sorting neurons based on autocorrelation to find nice signals

framerate = 1 / 2.004490057729
t = np.arange(1200) * framerate

mode_similarity = np.load('../Results/figure7_similarity_matrix.npy')
r_empirical = np.mean(np.diag(np.abs(mode_similarity)))
cutoff_empirical = identify_mode_cutoff_from_matrix_diagonal(mode_similarity)

In [ ]:
fig = PaperFigure(figsize=(7, 1.5), dpi=600)

fig.set_tick_length(1)
fig.set_font_size(6)
fig.add_background()

# Adding panels -------------------------------

for i in range(5):
    fig.add_axes(f'timeseries_{i}', (0, i * 0.25), 3, 0.25)
fig.add_axes('r_curve', (3.75, 0), 1.25, 1.25)
fig.add_axes('wavelengths', (7 - 1.25, 0), 1.25, 1.25)

fig.set_line_thickness(0.6)

# Content -------------------------------------

# Plotting example time series
nice_timeseries = [14, 17,26, 36, 39, 55]
for i in range(5):
    ax = fig.axes[f'timeseries_{i}']
    ax.plot(t, dff_tectum[order[nice_timeseries[i]]], color='black', linewidth=0.25)
    noise = np.random.normal(0, 1 * std, (dff_tectum.shape[1], ))
    ax.plot(t, dff_tectum[order[nice_timeseries[i]]] + noise, color='orange', linewidth=0.25, zorder=-10)
    noise = np.random.normal(0, 3 * std, (dff_tectum.shape[1], ))
    ax.plot(t, dff_tectum[order[nice_timeseries[i]]] + noise, color='red', linewidth=0.25, zorder=-20)

    #ax.spines[['top', 'right']].set_visible(False)
    ax.set_xlim([0, t[-1]])
    ax.set_ylim([-5, 5])
    ax.axis('off')

# Plotting geometry-function |r| vs noise level
ax = fig.axes['r_curve']
ax.plot(np.concatenate([[0], noise_levels / std]), np.concatenate([[r_empirical], np.mean(r_curve, axis=0)]), color='red', linewidth=1)
for i in range(r_curve.shape[1]):
    r_values = r_curve[:, i]
    ax.scatter([noise_levels[i] / std] * len(r_values), r_values, color='black', s=2, edgecolor='None')
ax.spines[['top', 'right']].set_visible(False)
ax.set_xticks([0, 1, 2, 3, 4, 5])
ax.set_xlim([0, 5.2])

# Plotting cutoff wavelength vs noise level
ax = fig.axes['wavelengths']
ax.plot(np.concatenate([[0], noise_levels / std]), np.concatenate([[wavelengths[cutoff_empirical]], np.mean(cutoff_wavelengths, axis=0)]), color='red', linewidth=1)
for i in range(cutoff_wavelengths.shape[1]):
    values = cutoff_wavelengths[:, i]
    ax.scatter([noise_levels[i] / std] * len(values), values, color='black', edgecolor='None', s=2)
ax.spines[['top', 'right']].set_visible(False)
ax.set_xticks([0, 1, 2, 3, 4, 5])
ax.set_xlim([0, 5.2])

# Displaying ----------------------------------

fig.save('../Figures/supp_noise_incomplete.svg')
fig.show()

#### Manually indicated values

In [ ]:
np.mean(cutoff_wavelengths[:, :6])

In [ ]:
np.std(cutoff_wavelengths[:, :6])

True cutoff wavelength

In [ ]:
wavelengths[cutoff_empirical]

In [ ]:
cutoff_empirical